TFG part 5- Sofía Valle López

First statistical analysis:

In [7]:
# ============================================================
# Analysis 1 — Direct comparison (ONE SHEET, NO GLOBAL)
# Fix: create Excel if it doesn't exist; append if it does.
# ============================================================

import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

INPUT_FILES = [
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window8.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window15.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window30.txt",
]
OUTPUT_XLSX = r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\stat_results.xlsx"
SHEET_NAME = "analysis1_direct_comparison"

COLS = [
    "patient","fragment","start_time","unit","window_size_s","num_windows",
    "time_peak_to_peak_amp","first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"
]

def ensure_dir_for(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)

def read_results(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, comment="#", header=None, names=COLS)
    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
    num = ["window_size_s","num_windows","time_peak_to_peak_amp",
           "first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"]
    df[num] = df[num].apply(pd.to_numeric, errors="coerce")
    return df

def bland_altman(x, y):
    diff = y - x
    bias = np.nanmean(diff)
    sd = np.nanstd(diff, ddof=1)
    return {"bias": bias, "loa_low": bias - 1.96*sd, "loa_high": bias + 1.96*sd, "sd_diff": sd}

def simple_linear_regression(x, y):
    mask = np.isfinite(x) & np.isfinite(y)
    x_fit, y_fit = x[mask], y[mask]
    if x_fit.size < 2:
        return {"intercept": np.nan, "slope": np.nan, "r2": np.nan}
    slope, intercept = np.polyfit(x_fit, y_fit, deg=1)
    y_pred = intercept + slope*x_fit
    ss_res = np.sum((y_fit - y_pred)**2)
    ss_tot = np.sum((y_fit - np.mean(y_fit))**2)
    r2 = 1 - ss_res/ss_tot if ss_tot != 0 else np.nan
    return {"intercept": intercept, "slope": slope, "r2": r2}

# --- Build tables ---
all_rows = []
summary_rows = []

for path in INPUT_FILES:
    df = read_results(path)
    win = int(df["window_size_s"].iloc[0]) if not df["window_size_s"].isna().all() else None
    window_label = f"{win}s" if win is not None else "unknown"

    print("\n" + "-"*80)
    print(f"Preview for file: {path}  |  window: {window_label}")
    print(df[["time_peak_to_peak_amp", "first_harmonic_p2p"]].head())

    ref = df["time_peak_to_peak_amp"]
    cand = df["first_harmonic_p2p"]

    rows = df[["patient","fragment","start_time","unit","window_size_s"]].copy()
    rows["time_peak_to_peak_amp"] = ref
    rows["first_harmonic_p2p"] = cand
    rows["abs_diff"] = (cand - ref).abs()
    rows["rel_error_pct"] = 100.0 * (cand - ref) / ref.replace(0, np.nan)
    rows["source_file"] = os.path.basename(path)
    rows["window_label"] = window_label
    all_rows.append(rows)

    valid = rows[["time_peak_to_peak_amp","first_harmonic_p2p"]].dropna()
    if len(valid) >= 2:
        from scipy.stats import pearsonr
        r, p = pearsonr(valid["time_peak_to_peak_amp"], valid["first_harmonic_p2p"])
        ba = bland_altman(valid["time_peak_to_peak_amp"].to_numpy(),
                          valid["first_harmonic_p2p"].to_numpy())
        reg = simple_linear_regression(valid["first_harmonic_p2p"].to_numpy(),
                                       valid["time_peak_to_peak_amp"].to_numpy())
    else:
        r = p = np.nan
        ba = {"bias": np.nan, "loa_low": np.nan, "loa_high": np.nan, "sd_diff": np.nan}
        reg = {"intercept": np.nan, "slope": np.nan, "r2": np.nan}

    summary_rows.append({
        "window_label": window_label,
        "n_pairs": len(valid),
        "pearson_r": r,
        "pearson_p_value": p,
        "bland_altman_bias": ba["bias"],
        "bland_altman_loa_low": ba["loa_low"],
        "bland_altman_loa_high": ba["loa_high"],
        "bland_altman_sd_diff": ba["sd_diff"],
        "mean_abs_diff": (valid["first_harmonic_p2p"] - valid["time_peak_to_peak_amp"]).abs().mean(),
        "median_abs_diff": (valid["first_harmonic_p2p"] - valid["time_peak_to_peak_amp"]).abs().median(),
        "mean_rel_error_pct": (100.0 * (valid["first_harmonic_p2p"] - valid["time_peak_to_peak_amp"])
                               / valid["time_peak_to_peak_amp"]).mean(),
        "linreg_intercept": reg["intercept"],
        "linreg_slope": reg["slope"],
        "linreg_r2": reg["r2"]
    })

all_rows_df = pd.concat(all_rows, ignore_index=True)
summary_by_window_df = pd.DataFrame(summary_rows)

# --- Write one sheet: create if missing, else append/replace sheet ---
ensure_dir_for(OUTPUT_XLSX)
excel_exists = os.path.exists(OUTPUT_XLSX)
mode = "a" if excel_exists else "w"

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl", mode=mode,
                    if_sheet_exists=("replace" if excel_exists else None)) as writer:
    all_rows_df.to_excel(writer, sheet_name=SHEET_NAME, index=False)
    startrow = all_rows_df.shape[0] + 2
    pd.DataFrame({"INFO": ["SUMMARY_BY_WINDOW (Analysis 1)"]}) \
        .to_excel(writer, sheet_name=SHEET_NAME, index=False, header=False, startrow=startrow)
    startrow += 2
    summary_by_window_df.to_excel(writer, sheet_name=SHEET_NAME, index=False, startrow=startrow)

print(f"\nSaved ONE sheet without global aggregation:\n  -> {OUTPUT_XLSX}  [sheet: {SHEET_NAME}]")



--------------------------------------------------------------------------------
Preview for file: C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window8.txt  |  window: 8s
   time_peak_to_peak_amp  first_harmonic_p2p
0                  0.800               0.517
1                  0.787               0.491
2                  0.727               0.493
3                  3.619               2.620
4                  3.376               2.682

--------------------------------------------------------------------------------
Preview for file: C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window15.txt  |  window: 15s
   time_peak_to_peak_amp  first_harmonic_p2p
0                  0.799               0.455
1                  0.786               0.423
2                  0.726               0.419
3                  3.638               2.506
4                  3.378               2.287

---------------------------------------------------------------

In [8]:
# ============================================================
# Visualizaciones principales por ventana (versión mejorada)
#   1) Pareado por fragmento (time vs first harmonic)
#   2) Scatter con correlación + identidad + regresión
#   3) Bland–Altman (freq - time)
# Fuente: TXT originales (no usa el Excel)
# ============================================================
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import pearsonr

# ------------------ RUTAS ------------------
INPUT_FILES = [
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window8.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window15.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window30.txt",
]
OUTPUT_DIR = r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\figures_MAIN"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------ ESTILO (grande y legible) ------------------
plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 18,
    "axes.titlesize": 24,
    "axes.labelsize": 20,
    "legend.fontsize": 18,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
})

# ------------------ COLUMNAS DE LOS .TXT ------------------
COLS = [
    "patient","fragment","start_time","unit","window_size_s","num_windows",
    "time_peak_to_peak_amp","first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"
]

# ------------------ HELPERS ------------------
def read_results_txt(path: str) -> pd.DataFrame:
    """Lee el .txt, ignora cabeceras con '#', tipa columnas y filtra pares válidos."""
    df = pd.read_csv(path, comment="#", header=None, names=COLS)
    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
    num = ["window_size_s","num_windows","time_peak_to_peak_amp",
           "first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"]
    df[num] = df[num].apply(pd.to_numeric, errors="coerce")
    df = df.dropna(subset=["time_peak_to_peak_amp", "first_harmonic_p2p"]).copy()
    return df

def label_from_window(df: pd.DataFrame, filename: str) -> str:
    """Etiqueta limpia del tipo '8s', '15s', '30s'."""
    if df["window_size_s"].notna().any():
        try:
            v = int(round(float(df["window_size_s"].dropna().iloc[0])))
            return f"{v}s"
        except Exception:
            pass
    for cand in ("8","15","30"):
        if cand in os.path.basename(filename):
            return f"{cand}s"
    return "unknown"

def simple_linear_regression(x, y):
    """Ajuste y = a + b·x. Devuelve (a, b, R²)."""
    x = np.asarray(x, float); y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if x.size < 2:
        return np.nan, np.nan, np.nan
    b, a = np.polyfit(x, y, 1)  # slope=b, intercept=a
    yhat = a + b*x
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = np.nan if ss_tot == 0 else 1 - ss_res/ss_tot
    return a, b, r2

def bland_altman_stats(x, y):
    """Devuelve (bias, sd, loa_low, loa_high) para diff = (y - x)."""
    x = np.asarray(x, float); y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    d = y[m] - x[m]
    if d.size == 0:
        return np.nan, np.nan, np.nan, np.nan
    bias = np.nanmean(d)
    sd = np.nanstd(d, ddof=1)
    return bias, sd, bias - 1.96*sd, bias + 1.96*sd

def add_textbox(fig, text, x=0.01, y=0.03):
    """Anotación legible en caja, pegada al borde inferior-izquierdo del lienzo."""
    fig.text(x, y, text, fontsize=16,
             bbox=dict(facecolor="white", alpha=0.95, edgecolor="black", boxstyle="round,pad=0.5"))

def place_legend_below(ax, ncol=2):
    """
    Coloca la leyenda fuera de la gráfica, centrada debajo.
    Aumenta el margen inferior para que no se corte.
    """
    # margen inferior amplio para que quepan leyenda + caja de texto
    plt.subplots_adjust(bottom=0.28)
    ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=True)

# ------------------ BUCLE PRINCIPAL ------------------
png_paths = []

for path in INPUT_FILES:
    df = read_results_txt(path)
    if df.empty:
        print(f"[AVISO] Sin datos válidos en: {path}")
        continue

    # Orden para el gráfico pareado
    df = df.sort_values(["start_time", "patient", "fragment"]).reset_index(drop=True)
    win_label = label_from_window(df, path)

    # Arrays
    x_idx = np.arange(len(df))           # índice de fragmentos
    y_time = df["time_peak_to_peak_amp"].to_numpy()
    y_freq = df["first_harmonic_p2p"].to_numpy()

    # ------------------ (1) PAREADO POR FRAGMENTO ------------------
    fig = plt.figure(figsize=(16, 9))
    ax = plt.gca()

    # unir pares
    for i in range(len(x_idx)):
        ax.plot([x_idx[i], x_idx[i]], [y_time[i], y_freq[i]], linewidth=1)

    # puntos (tiempo y primer armónico)
    ax.scatter(x_idx, y_time, label="Time P2P (mmHg)", s=70)
    ax.scatter(x_idx, y_freq, label="First harmonic P2P (mmHg)", s=70)

    ax.set_xlabel("Fragments ordered by start_time")
    ax.set_ylabel("Amplitude (mmHg)")
    ax.set_title(f"Paired comparison per fragment — {win_label}")
    ax.grid(True, alpha=0.3)

    # leyenda fuera, centrada abajo
    place_legend_below(ax, ncol=2)

    # métricas (caja de texto)
    abs_diff = np.abs(y_freq - y_time)
    rel_err = 100.0 * (y_freq - y_time) / np.where(y_time == 0, np.nan, y_time)
    bias, sd, loa_low, loa_high = bland_altman_stats(y_time, y_freq)
    txt = (
        f"Mean |diff| = {np.nanmean(abs_diff):.3f} mmHg    "
        f"Median |diff| = {np.nanmedian(abs_diff):.3f} mmHg\n"
        f"Mean rel. error = {np.nanmean(rel_err):.1f}%    "
        f"Bias = {bias:.3f} mmHg,  LoA = [{loa_low:.3f}, {loa_high:.3f}] mmHg"
    )
    add_textbox(fig, txt, x=0.02, y=0.04)

    f1 = os.path.join(OUTPUT_DIR, f"01_paired_per_fragment_{win_label}.png")
    plt.savefig(f1, bbox_inches="tight"); plt.close()
    png_paths.append(f1)

    # ------------------ (2) SCATTER + IDENTIDAD + REGRESIÓN ------------------
    mask = np.isfinite(y_time) & np.isfinite(y_freq)
    r, pval = (np.nan, np.nan)
    if mask.sum() >= 2:
        r, pval = pearsonr(y_time[mask], y_freq[mask])

    a, b, r2 = simple_linear_regression(y_time, y_freq)
    xmax = float(np.nanmax(y_time)); ymax = float(np.nanmax(y_freq))
    lim = max(xmax, ymax, 0.0)

    fig = plt.figure(figsize=(16, 9))
    ax = plt.gca()
    ax.scatter(y_time, y_freq, s=70)
    # identidad
    ax.plot([0, lim], [0, lim], linestyle="--", linewidth=1)
    # regresión
    if not (math.isnan(a) or math.isnan(b)):
        xline = np.linspace(0, lim, 200)
        yline = a + b * xline
        ax.plot(xline, yline, linestyle="-", linewidth=1)

    ax.set_xlabel("Time peak-to-peak amplitude (mmHg)")
    ax.set_ylabel("First harmonic P2P (mmHg)")
    ax.set_title(f"Time vs First Harmonic P2P — {win_label}")
    ax.grid(True, alpha=0.3)

    # leyenda (opcional); si la añades, también la mando abajo
    # ax.legend(["Identity", "Regression"], loc="upper center", bbox_to_anchor=(0.5, -0.18), ncol=2)

    # caja de estadísticas
    txt = (f"Pearson r = {r:.3f},  p = {pval:.3g}\n"
           f"Linear fit: y = a + b·x    a = {a:.3f},  b = {b:.3f}\n"
           f"R² = {r2:.3f}")
    add_textbox(fig, txt, x=0.02, y=0.04)

    f2 = os.path.join(OUTPUT_DIR, f"02_scatter_time_vs_firstharm_{win_label}.png")
    plt.savefig(f2, bbox_inches="tight"); plt.close()
    png_paths.append(f2)

    # ------------------ (3) BLAND–ALTMAN ------------------
    bias, sd, loa_low, loa_high = bland_altman_stats(y_time, y_freq)
    mean_xy = (y_time + y_freq) / 2.0
    diff = y_freq - y_time

    fig = plt.figure(figsize=(16, 9))
    ax = plt.gca()
    ax.scatter(mean_xy, diff, s=70)
    ax.axhline(bias, linestyle="-", linewidth=1)
    ax.axhline(loa_low, linestyle="--", linewidth=1)
    ax.axhline(loa_high, linestyle="--", linewidth=1)

    ax.set_xlabel("Mean of methods (mmHg)")
    ax.set_ylabel("Difference (freq - time) (mmHg)")
    ax.set_title(f"Bland–Altman — {win_label}")
    ax.grid(True, alpha=0.3)

    # caja de estadísticas
    txt = (f"Bias = {bias:.3f} mmHg     SD = {sd:.3f} mmHg\n"
           f"LoA = [{loa_low:.3f}, {loa_high:.3f}] mmHg")
    add_textbox(fig, txt, x=0.02, y=0.04)

    f3 = os.path.join(OUTPUT_DIR, f"03_bland_altman_{win_label}.png")
    plt.savefig(f3, bbox_inches="tight"); plt.close()
    png_paths.append(f3)

# ------------------ PDF MULTIPÁGINA ------------------
pdf_path = os.path.join(OUTPUT_DIR, "main_visuals_by_window.pdf")
with PdfPages(pdf_path) as pdf:
    for p in png_paths:
        img = plt.imread(p)
        plt.figure(figsize=(11.69, 8.27))  # A4 apaisado
        plt.imshow(img)
        plt.axis("off")
        pdf.savefig(bbox_inches="tight")
        plt.close()

print("Listo. Figuras guardadas en:", OUTPUT_DIR)
print("PDF multipágina:", pdf_path)


Listo. Figuras guardadas en: C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\figures_MAIN
PDF multipágina: C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\figures_MAIN\main_visuals_by_window.pdf


Second statistical analysis

In [14]:
# ============================================================
# Analysis 2 — Energy ratio (multi/first) con resumen abajo
# -> Usa if_sheet_exists="overlay" para escribir dos veces en la misma hoja
# ============================================================
import os
import numpy as np
import pandas as pd
from openpyxl import load_workbook

INPUT_FILES = [
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window8.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window15.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window30.txt",
]
OUTPUT_XLSX = r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\stat_results.xlsx"
SHEET_NAME = "analysis2_energy_ratio"

COLS = [
    "patient","fragment","start_time","unit","window_size_s","num_windows",
    "time_peak_to_peak_amp","first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"
]

def ensure_dir_for(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)

def read_results(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, comment="#", header=None, names=COLS)
    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
    num_cols = ["window_size_s","num_windows","time_peak_to_peak_amp",
                "first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"]
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
    return df

# ---------- Cálculo ----------
all_rows, summary_rows = [], []

for path in INPUT_FILES:
    df = read_results(path)
    win = int(df["window_size_s"].iloc[0]) if not df["window_size_s"].isna().all() else None
    window_label = f"{win}s" if win is not None else "unknown"

    print("\n" + "-"*80)
    print(f"Preview for file: {os.path.basename(path)} | window: {window_label}")
    print(df[["multi_harmonics_peak", "first_harmonic_peak"]].head())

    ratio = df["multi_harmonics_peak"] / df["first_harmonic_peak"].replace(0, np.nan)

    rows = df[["patient","fragment","start_time","unit","window_size_s"]].copy()
    rows["multi_harmonics_peak"] = df["multi_harmonics_peak"]
    rows["first_harmonic_peak"] = df["first_harmonic_peak"]
    rows["energy_ratio_R"]      = ratio
    rows["source_file"]         = os.path.basename(path)
    rows["window_label"]        = window_label
    all_rows.append(rows)

    valid_ratio = ratio.dropna()
    summary_rows.append({
        "window_label": window_label,
        "n_pairs": len(valid_ratio),
        "mean_ratio": np.nanmean(valid_ratio),
        "median_ratio": np.nanmedian(valid_ratio),
        "std_ratio": np.nanstd(valid_ratio, ddof=1),
        "min_ratio": np.nanmin(valid_ratio),
        "max_ratio": np.nanmax(valid_ratio),
        "q25_ratio": np.nanpercentile(valid_ratio, 25),
        "q75_ratio": np.nanpercentile(valid_ratio, 75),
    })

all_rows_df = pd.concat(all_rows, ignore_index=True)
summary_df   = pd.DataFrame(summary_rows)

# ---------- Escritura: detalle + título + resumen ----------
ensure_dir_for(OUTPUT_XLSX)

# Borra la hoja si ya existe para empezar limpia
if os.path.exists(OUTPUT_XLSX):
    wb = load_workbook(OUTPUT_XLSX)
    if SHEET_NAME in wb.sheetnames:
        wb.remove(wb[SHEET_NAME])
        wb.save(OUTPUT_XLSX)
    wb.close()
    mode = "a"
else:
    mode = "w"

# 👇 CLAVE: if_sheet_exists="overlay" permite escribir otra vez en la misma hoja
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl", mode=mode, if_sheet_exists="overlay") as writer:
    # 1) Detalle
    all_rows_df.to_excel(writer, sheet_name=SHEET_NAME, index=False)

    # 2) Título + Resumen con su propio header, más abajo
    ws = writer.sheets[SHEET_NAME]
    startrow = all_rows_df.shape[0] + 2   # deja una línea en blanco
    ws.cell(row=startrow, column=1, value="SUMMARY_BY_WINDOW (Analysis 2)")
    # tabla de resumen (con encabezados) dos filas debajo del título
    summary_df.to_excel(writer, sheet_name=SHEET_NAME, index=False, startrow=startrow + 2)

print(f"\nGuardado en: {OUTPUT_XLSX}  [sheet: {SHEET_NAME}]")



--------------------------------------------------------------------------------
Preview for file: final_results_window8.txt | window: 8s
   multi_harmonics_peak  first_harmonic_peak
0                 0.271                0.259
1                 0.263                0.246
2                 0.261                0.247
3                 1.381                1.310
4                 1.392                1.341

--------------------------------------------------------------------------------
Preview for file: final_results_window15.txt | window: 15s
   multi_harmonics_peak  first_harmonic_peak
0                 0.237                0.227
1                 0.222                0.211
2                 0.220                0.209
3                 1.317                1.253
4                 1.197                1.143

--------------------------------------------------------------------------------
Preview for file: final_results_window30.txt | window: 30s
   multi_harmonics_peak  first_harmonic

In [17]:
# ============================================================
# Visualizaciones del Análisis 2 (por ventana)
#   Para cada tamaño de ventana (8s, 15s, 30s) crea:
#     (1) Histograma del ratio R = multi / first
#     (2) Dispersión: first_harmonic_peak vs multi_harmonics_peak
# ============================================================
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# ------------------ RUTAS ------------------
INPUT_FILES = [
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window8.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window15.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window30.txt",
]
OUTPUT_DIR = r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\figures_ANALYSIS2"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------ COLUMNAS DE LOS .TXT ------------------
COLS = [
    "patient","fragment","start_time","unit","window_size_s","num_windows",
    "time_peak_to_peak_amp","first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"
]

# ------------------ ESTILO ------------------
plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 18,
    "axes.titlesize": 24,
    "axes.labelsize": 20,
    "legend.fontsize": 16,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
})

# ------------------ HELPERS ------------------
def read_results_txt(path: str) -> pd.DataFrame:
    """Lee el .txt, tipa columnas y devuelve DataFrame con NaNs donde toque."""
    df = pd.read_csv(path, comment="#", header=None, names=COLS)
    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
    num = ["window_size_s","num_windows","time_peak_to_peak_amp",
           "first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"]
    df[num] = df[num].apply(pd.to_numeric, errors="coerce")
    return df

def label_from_window(df: pd.DataFrame, filename: str) -> str:
    """Etiqueta limpia tipo '8s', '15s', '30s'."""
    if df["window_size_s"].notna().any():
        try:
            v = int(round(float(df["window_size_s"].dropna().iloc[0])))
            return f"{v}s"
        except Exception:
            pass
    for cand in ("8","15","30"):
        if cand in os.path.basename(filename):
            return f"{cand}s"
    return "unknown"

def add_textbox(fig, text, x=0.02, y=0.04):
    """Caja de texto pegada a la esquina inferior-izquierda."""
    fig.text(x, y, text, fontsize=14,
             bbox=dict(facecolor="white", alpha=0.95, edgecolor="black", boxstyle="round,pad=0.5"))

def simple_linear_regression(x, y):
    """Ajuste y = a + b·x. Devuelve (a, b, R²)."""
    x = np.asarray(x, float); y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if x.size < 2:
        return np.nan, np.nan, np.nan
    b, a = np.polyfit(x, y, 1)  # slope=b, intercept=a
    yhat = a + b*x
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = np.nan if ss_tot == 0 else 1 - ss_res/ss_tot
    return a, b, r2

# ------------------ BUCLE PRINCIPAL ------------------
for path in INPUT_FILES:
    df = read_results_txt(path)
    win_label = label_from_window(df, path)

    # Preview para verificar columnas correctas
    print("\n" + "-"*80)
    print(f"File: {os.path.basename(path)} | window: {win_label}")
    print(df[["multi_harmonics_peak", "first_harmonic_peak"]].head())

    # Datos
    first = df["first_harmonic_peak"].to_numpy(dtype=float)
    multi = df["multi_harmonics_peak"].to_numpy(dtype=float)
    ratio = multi / np.where(first == 0, np.nan, first)

    # ------------------ (1) HISTOGRAMA DE R ------------------
    valid_R = ratio[np.isfinite(ratio)]
    if valid_R.size > 0:
        fig = plt.figure(figsize=(14, 8))
        ax = plt.gca()

        # Histograma
        bins = max(10, int(np.sqrt(valid_R.size)))
        ax.hist(valid_R, bins=bins, alpha=0.85, label="Ratio R")

        # Estadísticos
        mean_R   = float(np.nanmean(valid_R))
        median_R = float(np.nanmedian(valid_R))
        q25      = float(np.nanpercentile(valid_R, 25))
        q75      = float(np.nanpercentile(valid_R, 75))
        std_R    = float(np.nanstd(valid_R, ddof=1))

        # Líneas guía
        ax.axvline(1.0, linestyle="--", linewidth=1, label="Identity R=1")
        ax.axvline(mean_R, linestyle="-", linewidth=1, label="Mean")
        ax.axvline(median_R, linestyle=":", linewidth=1, label="Median")

        ax.set_xlabel("Energy ratio  R = multi_harmonics_peak / first_harmonic_peak")
        ax.set_ylabel("Count")
        ax.set_title(f"Distribution of R — {win_label}")
        ax.grid(True, alpha=0.3)

        txt = (f"n = {valid_R.size}\n"
               f"mean = {mean_R:.3f}   median = {median_R:.3f}\n"
               f"std = {std_R:.3f}   Q25 = {q25:.3f}   Q75 = {q75:.3f}")
        add_textbox(fig, txt)

        # 👇 Leyenda fuera y centrada abajo
        ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=True)
        plt.subplots_adjust(bottom=0.28)

        out_png = os.path.join(OUTPUT_DIR, f"hist_ratio_R_{win_label}.png")
        plt.savefig(out_png, bbox_inches="tight"); plt.close()
        print("Saved:", out_png)

    # ------------------ (2) DISPERSIÓN multi vs first ------------------
    mask = np.isfinite(first) & np.isfinite(multi)
    xf = first[mask]; yf = multi[mask]

    if xf.size >= 2:
        r, pval = pearsonr(xf, yf)
        a, b, r2 = simple_linear_regression(xf, yf)

        lim = float(np.nanmax([xf.max(), yf.max(), 0.0])) * 1.05

        fig = plt.figure(figsize=(14, 8))
        ax = plt.gca()

        ax.scatter(xf, yf, s=70, label="Fragments")
        # Línea identidad
        ax.plot([0, lim], [0, lim], linestyle="--", linewidth=1, label="Identity y=x")
        # Línea regresión
        if not (math.isnan(a) or math.isnan(b)):
            xline = np.linspace(0, lim, 200)
            yline = a + b * xline
            ax.plot(xline, yline, linestyle="-", linewidth=1, label="Linear fit")

        ax.set_xlabel("First harmonic peak (mmHg)")
        ax.set_ylabel("Multi-harmonics peak (mmHg)")
        ax.set_title(f"multi vs first — {win_label}")
        ax.grid(True, alpha=0.3)

        # 👇 Leyenda fuera, centrada abajo
        ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=True)
        plt.subplots_adjust(bottom=0.28)

        txt = (f"Pearson r = {r:.3f},  p = {pval:.3g}\n"
               f"Fit: y = a + b·x   a = {a:.3f},  b = {b:.3f}\n"
               f"R² = {r2:.3f}")
        add_textbox(fig, txt)

        out_png = os.path.join(OUTPUT_DIR, f"scatter_multi_vs_first_{win_label}.png")
        plt.savefig(out_png, bbox_inches="tight"); plt.close()
        print("Saved:", out_png)

print("\nListo. Revisa la carpeta:", OUTPUT_DIR)



--------------------------------------------------------------------------------
File: final_results_window8.txt | window: 8s
   multi_harmonics_peak  first_harmonic_peak
0                 0.271                0.259
1                 0.263                0.246
2                 0.261                0.247
3                 1.381                1.310
4                 1.392                1.341
Saved: C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\figures_ANALYSIS2\hist_ratio_R_8s.png
Saved: C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\figures_ANALYSIS2\scatter_multi_vs_first_8s.png

--------------------------------------------------------------------------------
File: final_results_window15.txt | window: 15s
   multi_harmonics_peak  first_harmonic_peak
0                 0.237                0.227
1                 0.222                0.211
2                 0.220                0.209
3                 1.317                1.253
4                 1.197              

Third analysis:

In [19]:
# ============================================================
# Analysis 3 — Compare (2 * multi_harmonics_peak) vs time P2P
#   - Uses the three TXT inputs (8s, 15s, 30s)
#   - Writes to the same Excel as Analyses 1 & 2, new sheet
#   - Prints previews to verify correct columns
# ============================================================
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from openpyxl import load_workbook

# ------------------ INPUT / OUTPUT ------------------
INPUT_FILES = [
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window8.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window15.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window30.txt",
]
OUTPUT_XLSX = r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\stat_results.xlsx"
SHEET_NAME = "analysis3_multiX2_vs_time"

# Columns present in the TXT results
COLS = [
    "patient","fragment","start_time","unit","window_size_s","num_windows",
    "time_peak_to_peak_amp","first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"
]

# ------------------ HELPERS ------------------
def ensure_dir_for(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)

def read_results(path: str) -> pd.DataFrame:
    """Read the results .txt, coerce dtypes, keep NaNs if any."""
    df = pd.read_csv(path, comment="#", header=None, names=COLS)
    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
    num_cols = [
        "window_size_s","num_windows","time_peak_to_peak_amp",
        "first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"
    ]
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
    # Order rows to keep outputs stable and readable
    df = df.sort_values(["patient","fragment","start_time"]).reset_index(drop=True)
    return df

def bland_altman(x, y):
    """Return BA stats for diff = (y - x)."""
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    m = np.isfinite(x) & np.isfinite(y)
    d = y[m] - x[m]
    if d.size == 0:
        return {"bias": np.nan, "sd": np.nan, "loa_low": np.nan, "loa_high": np.nan}
    bias = float(np.nanmean(d))
    sd   = float(np.nanstd(d, ddof=1))
    return {
        "bias": bias,
        "sd": sd,
        "loa_low": bias - 1.96*sd,
        "loa_high": bias + 1.96*sd
    }

def simple_linear_regression(x, y):
    """
    Fit y = a + b*x on finite pairs.
    Returns dict(intercept=a, slope=b, r2).
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if x.size < 2:
        return {"intercept": np.nan, "slope": np.nan, "r2": np.nan}
    b, a = np.polyfit(x, y, 1)  # slope=b, intercept=a
    yhat = a + b*x
    ss_res = float(np.sum((y - yhat)**2))
    ss_tot = float(np.sum((y - np.nanmean(y))**2))
    r2 = np.nan if ss_tot == 0 else 1 - ss_res/ss_tot
    return {"intercept": float(a), "slope": float(b), "r2": float(r2)}

# ------------------ MAIN LOOP ------------------
all_rows = []
summary_rows = []

for path in INPUT_FILES:
    df = read_results(path)

    # Window label (e.g., '8s', '15s', '30s')
    win = int(df["window_size_s"].iloc[0]) if not df["window_size_s"].isna().all() else None
    window_label = f"{win}s" if win is not None else "unknown"

    # Reference (time) and candidate (frequency-based estimate)
    ref_time = df["time_peak_to_peak_amp"]
    cand_multi_x2 = 2.0 * df["multi_harmonics_peak"]

    # ------- PREVIEW: show first rows to verify we picked right columns -------
    print("\n" + "-"*90)
    print(f"File: {os.path.basename(path)} | window: {window_label}")
    preview = pd.DataFrame({
        "patient": df["patient"].head(5),
        "fragment": df["fragment"].head(5),
        "time_peak_to_peak_amp": ref_time.head(5),
        "multi_harmonics_peak": df["multi_harmonics_peak"].head(5),
        "candidate_multi_x2": cand_multi_x2.head(5)
    })
    print(preview.to_string(index=False))

    # ------- Row-wise metrics -------
    rows = df[["patient","fragment","start_time","unit","window_size_s"]].copy()
    rows["time_peak_to_peak_amp"] = ref_time
    rows["candidate_multi_x2"]    = cand_multi_x2
    rows["abs_diff"] = (cand_multi_x2 - ref_time).abs()
    rows["rel_error_pct"] = 100.0 * (cand_multi_x2 - ref_time) / ref_time.replace(0, np.nan)
    rows["source_file"]  = os.path.basename(path)
    rows["window_label"] = window_label
    all_rows.append(rows)

    # ------- Summary-by-window (valid pairs only) -------
    valid = rows[["time_peak_to_peak_amp","candidate_multi_x2"]].dropna()
    if len(valid) >= 2:
        r, p = pearsonr(valid["time_peak_to_peak_amp"], valid["candidate_multi_x2"])
        ba   = bland_altman(valid["time_peak_to_peak_amp"].to_numpy(),
                            valid["candidate_multi_x2"].to_numpy())
        reg  = simple_linear_regression(valid["candidate_multi_x2"].to_numpy(),
                                        valid["time_peak_to_peak_amp"].to_numpy())
    else:
        r = p = np.nan
        ba = {"bias": np.nan, "sd": np.nan, "loa_low": np.nan, "loa_high": np.nan}
        reg = {"intercept": np.nan, "slope": np.nan, "r2": np.nan}

    summary_rows.append({
        "window_label": window_label,
        "n_pairs": len(valid),
        "pearson_r": r,
        "pearson_p_value": p,
        "bland_altman_bias_mmHg": ba["bias"],
        "bland_altman_sd_diff_mmHg": ba["sd"],
        "bland_altman_loa_low_mmHg": ba["loa_low"],
        "bland_altman_loa_high_mmHg": ba["loa_high"],
        "mean_abs_diff_mmHg": (valid["candidate_multi_x2"] - valid["time_peak_to_peak_amp"]).abs().mean(),
        "median_abs_diff_mmHg": (valid["candidate_multi_x2"] - valid["time_peak_to_peak_amp"]).abs().median(),
        "mean_rel_error_pct": (
            100.0 * (valid["candidate_multi_x2"] - valid["time_peak_to_peak_amp"])
            / valid["time_peak_to_peak_amp"]
        ).mean(),
        "linreg_intercept": reg["intercept"],
        "linreg_slope": reg["slope"],
        "linreg_r2": reg["r2"],
    })

# ------------------ BUILD TABLES ------------------
all_rows_df        = pd.concat(all_rows, ignore_index=True)
summary_by_window  = pd.DataFrame(summary_rows)

# ------------------ WRITE EXCEL (new sheet, clean) ------------------
ensure_dir_for(OUTPUT_XLSX)

# If workbook exists and the sheet already exists, remove it to start clean
if os.path.exists(OUTPUT_XLSX):
    wb = load_workbook(OUTPUT_XLSX)
    if SHEET_NAME in wb.sheetnames:
        ws = wb[SHEET_NAME]
        wb.remove(ws)
        wb.save(OUTPUT_XLSX)
    wb.close()
    mode = "a"
else:
    mode = "w"

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl", mode=mode, if_sheet_exists="overlay") as writer:
    # Detailed rows
    all_rows_df.to_excel(writer, sheet_name=SHEET_NAME, index=False)

    # Title + summary placed below the detail table
    ws = writer.sheets[SHEET_NAME]
    startrow = all_rows_df.shape[0] + 2  # one blank line
    ws.cell(row=startrow, column=1, value="SUMMARY_BY_WINDOW (Analysis 3)")
    # Summary table two lines below the title
    summary_by_window.to_excel(writer, sheet_name=SHEET_NAME, index=False, startrow=startrow + 2)

print(f"\nSaved in: {OUTPUT_XLSX}  [sheet: {SHEET_NAME}]")



------------------------------------------------------------------------------------------
File: final_results_window8.txt | window: 8s
   patient   fragment  time_peak_to_peak_amp  multi_harmonics_peak  candidate_multi_x2
 paciente1 fragmento1                  0.800                 0.271               0.542
 paciente1 fragmento2                  0.787                 0.263               0.526
 paciente1 fragmento3                  0.727                 0.261               0.522
paciente10 fragmento1                  6.255                 2.033               4.066
paciente10 fragmento2                 10.456                 3.642               7.284

------------------------------------------------------------------------------------------
File: final_results_window15.txt | window: 15s
   patient   fragment  time_peak_to_peak_amp  multi_harmonics_peak  candidate_multi_x2
 paciente1 fragmento1                  0.799                 0.237               0.474
 paciente1 fragmento2       

In [20]:
# ============================================================
# Analysis 3 — Visualizations
#   Compare: Time P2P (reference) vs Candidate = 2 * multi_harmonics_peak
#   For each window (8s, 15s, 30s) it creates:
#     (1) Paired per fragment
#     (2) Scatter with identity + linear regression
#     (3) Bland–Altman (candidate - reference)
#   Sources: original TXT files (no Excel needed)
# ============================================================
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import pearsonr

# ------------------ PATHS ------------------
INPUT_FILES = [
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window8.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window15.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window30.txt",
]
OUTPUT_DIR = r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\figures_ANALYSIS3"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------ STYLE (large and readable) ------------------
plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 18,
    "axes.titlesize": 24,
    "axes.labelsize": 20,
    "legend.fontsize": 18,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
})

# ------------------ TXT COLUMNS ------------------
COLS = [
    "patient","fragment","start_time","unit","window_size_s","num_windows",
    "time_peak_to_peak_amp","first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"
]

# ------------------ HELPERS ------------------
def read_results_txt(path: str) -> pd.DataFrame:
    """Read TXT, coerce dtypes, keep NaNs if any, and sort for stable plots."""
    df = pd.read_csv(path, comment="#", header=None, names=COLS)
    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
    num = ["window_size_s","num_windows","time_peak_to_peak_amp",
           "first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"]
    df[num] = df[num].apply(pd.to_numeric, errors="coerce")
    df = df.sort_values(["start_time","patient","fragment"]).reset_index(drop=True)
    return df

def label_from_window(df: pd.DataFrame, filename: str) -> str:
    """Return clean label like '8s', '15s', '30s'."""
    if df["window_size_s"].notna().any():
        try:
            v = int(round(float(df["window_size_s"].dropna().iloc[0])))
            return f"{v}s"
        except Exception:
            pass
    for cand in ("8","15","30"):
        if cand in os.path.basename(filename):
            return f"{cand}s"
    return "unknown"

def add_textbox(fig, text, x=0.02, y=0.04):
    """Readable annotation box near lower-left corner."""
    fig.text(x, y, text, fontsize=16,
             bbox=dict(facecolor="white", alpha=0.95, edgecolor="black", boxstyle="round,pad=0.5"))

def place_legend_below(ax, ncol=2):
    """Place legend outside, centered below, with extra bottom margin."""
    plt.subplots_adjust(bottom=0.28)
    ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=True)

def bland_altman_stats(x, y):
    """Return (bias, sd, loa_low, loa_high) for diff = (y - x)."""
    x = np.asarray(x, float); y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    d = y[m] - x[m]
    if d.size == 0:
        return np.nan, np.nan, np.nan, np.nan
    bias = float(np.nanmean(d))
    sd   = float(np.nanstd(d, ddof=1))
    return bias, sd, bias - 1.96*sd, bias + 1.96*sd

def simple_linear_regression(x, y):
    """Fit y = a + b·x. Return (a, b, R²)."""
    x = np.asarray(x, float); y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if x.size < 2:
        return np.nan, np.nan, np.nan
    b, a = np.polyfit(x, y, 1)  # slope=b, intercept=a
    yhat = a + b*x
    ss_res = float(np.sum((y - yhat)**2))
    ss_tot = float(np.sum((y - np.mean(y))**2))
    r2 = np.nan if ss_tot == 0 else 1 - ss_res/ss_tot
    return a, b, r2

# ------------------ MAIN LOOP ------------------
png_paths = []

for path in INPUT_FILES:
    df = read_results_txt(path)
    win_label = label_from_window(df, path)

    # Data
    time_p2p = df["time_peak_to_peak_amp"].to_numpy(dtype=float)           # reference
    multi    = df["multi_harmonics_peak"].to_numpy(dtype=float)
    cand     = 2.0 * multi                                                # candidate = 2 * multi_harmonics_peak

    # ------- Preview: first rows to verify columns -------
    print("\n" + "-"*90)
    print(f"File: {os.path.basename(path)} | window: {win_label}")
    print(pd.DataFrame({
        "patient":  df["patient"].head(5),
        "fragment": df["fragment"].head(5),
        "time_P2P": df["time_peak_to_peak_amp"].head(5),
        "multi":    df["multi_harmonics_peak"].head(5),
        "candidate_multi_x2": (2.0 * df["multi_harmonics_peak"]).head(5)
    }).to_string(index=False))

    # ------------------ (1) PAIRED PER FRAGMENT ------------------
    x_idx = np.arange(len(df))
    fig = plt.figure(figsize=(16, 9))
    ax = plt.gca()

    # connect pairs
    for i in range(len(x_idx)):
        ax.plot([x_idx[i], x_idx[i]], [time_p2p[i], cand[i]], linewidth=1)

    ax.scatter(x_idx, time_p2p, s=70, label="Time P2P (mmHg)")
    ax.scatter(x_idx, cand,     s=70, label="Candidate: 2 × multi (mmHg)")

    ax.set_xlabel("Fragments ordered by start_time")
    ax.set_ylabel("Amplitude (mmHg)")
    ax.set_title(f"Paired comparison per fragment — {win_label}")
    ax.grid(True, alpha=0.3)

    place_legend_below(ax, ncol=2)

    # stats box
    abs_diff = np.abs(cand - time_p2p)
    rel_err  = 100.0 * (cand - time_p2p) / np.where(time_p2p == 0, np.nan, time_p2p)
    bias, sd, loa_low, loa_high = bland_altman_stats(time_p2p, cand)
    txt = (f"Mean |diff| = {np.nanmean(abs_diff):.3f} mmHg   "
           f"Median |diff| = {np.nanmedian(abs_diff):.3f} mmHg\n"
           f"Mean rel. error = {np.nanmean(rel_err):.1f}%   "
           f"Bias = {bias:.3f} mmHg,  LoA = [{loa_low:.3f}, {loa_high:.3f}] mmHg")
    add_textbox(fig, txt, x=0.02, y=0.04)

    out1 = os.path.join(OUTPUT_DIR, f"01_paired_per_fragment_multiX2_{win_label}.png")
    plt.savefig(out1, bbox_inches="tight"); plt.close(); png_paths.append(out1)

    # ------------------ (2) SCATTER + IDENTITY + REGRESSION ------------------
    mask = np.isfinite(time_p2p) & np.isfinite(cand)
    r, pval = (np.nan, np.nan)
    if mask.sum() >= 2:
        r, pval = pearsonr(time_p2p[mask], cand[mask])
        a, b, r2 = simple_linear_regression(time_p2p, cand)
    else:
        a = b = r2 = np.nan

    lim = float(np.nanmax([np.nanmax(time_p2p), np.nanmax(cand), 0.0]))
    lim = lim * 1.05 if np.isfinite(lim) and lim > 0 else 1.0

    fig = plt.figure(figsize=(16, 9))
    ax = plt.gca()

    ax.scatter(time_p2p, cand, s=70)
    # identity
    ax.plot([0, lim], [0, lim], linestyle="--", linewidth=1, label="Identity")
    # regression
    if not (math.isnan(a) or math.isnan(b)):
        xline = np.linspace(0, lim, 200)
        yline = a + b * xline
        ax.plot(xline, yline, linestyle="-", linewidth=1, label="Linear fit")

    ax.set_xlabel("Time peak-to-peak amplitude (mmHg)")
    ax.set_ylabel("Candidate: 2 × multi_harmonics_peak (mmHg)")
    ax.set_title(f"Time vs Candidate (2×multi) — {win_label}")
    ax.grid(True, alpha=0.3)
    place_legend_below(ax, ncol=2)

    txt = (f"Pearson r = {r:.3f},  p = {pval:.3g}\n"
           f"Fit: y = a + b·x   a = {a:.3f},  b = {b:.3f}\n"
           f"R² = {r2:.3f}")
    add_textbox(fig, txt, x=0.02, y=0.04)

    out2 = os.path.join(OUTPUT_DIR, f"02_scatter_time_vs_multiX2_{win_label}.png")
    plt.savefig(out2, bbox_inches="tight"); plt.close(); png_paths.append(out2)

    # ------------------ (3) BLAND–ALTMAN ------------------
    bias, sd, loa_low, loa_high = bland_altman_stats(time_p2p, cand)
    mean_xy = (time_p2p + cand) / 2.0
    diff    = cand - time_p2p

    fig = plt.figure(figsize=(16, 9))
    ax = plt.gca()
    ax.scatter(mean_xy, diff, s=70)
    ax.axhline(bias,     linestyle="-",  linewidth=1, label=f"Bias {bias:.3f}")
    ax.axhline(loa_low,  linestyle="--", linewidth=1, label=f"LoA low {loa_low:.3f}")
    ax.axhline(loa_high, linestyle="--", linewidth=1, label=f"LoA high {loa_high:.3f}")

    ax.set_xlabel("Mean of methods (mmHg)")
    ax.set_ylabel("Difference (candidate - time) (mmHg)")
    ax.set_title(f"Bland–Altman — {win_label}")
    ax.grid(True, alpha=0.3)
    place_legend_below(ax, ncol=3)

    txt = (f"Bias = {bias:.3f} mmHg   SD = {sd:.3f} mmHg\n"
           f"LoA = [{loa_low:.3f}, {loa_high:.3f}] mmHg")
    add_textbox(fig, txt, x=0.02, y=0.04)

    out3 = os.path.join(OUTPUT_DIR, f"03_bland_altman_multiX2_{win_label}.png")
    plt.savefig(out3, bbox_inches="tight"); plt.close(); png_paths.append(out3)

# ------------------ MULTI-PAGE PDF ------------------
pdf_path = os.path.join(OUTPUT_DIR, "analysis3_visuals_by_window.pdf")
with PdfPages(pdf_path) as pdf:
    for p in png_paths:
        img = plt.imread(p)
        plt.figure(figsize=(11.69, 8.27))  # A4 landscape
        plt.imshow(img)
        plt.axis("off")
        pdf.savefig(bbox_inches="tight")
        plt.close()

print("Done. Figures saved in:", OUTPUT_DIR)
print("PDF multipage:", pdf_path)



------------------------------------------------------------------------------------------
File: final_results_window8.txt | window: 8s
   patient   fragment  time_P2P  multi  candidate_multi_x2
paciente20 fragmento1     4.850  1.635               3.270
paciente20 fragmento2     5.910  1.929               3.858
paciente20 fragmento3     5.014  1.761               3.522
 paciente3 fragmento1     8.480  3.344               6.688
 paciente3 fragmento2     8.189  3.417               6.834

------------------------------------------------------------------------------------------
File: final_results_window15.txt | window: 15s
   patient   fragment  time_P2P  multi  candidate_multi_x2
paciente20 fragmento1     4.835  1.486               2.972
paciente20 fragmento2     5.877  1.932               3.864
paciente20 fragmento3     5.016  1.711               3.422
 paciente3 fragmento1     8.635  3.085               6.170
 paciente3 fragmento2     8.191  3.227               6.454

---------------

Fourth analysis

In [21]:
# ============================================================
# Analysis 4 — Correlation matrix + strong-pair scatters
#   - Works per window (8s, 15s, 30s) from your TXT sources
#   - Metrics: time_peak_to_peak_amp, first_harmonic_peak,
#              first_harmonic_p2p, multi_harmonics_peak
#   - Optional EXTENDED mode adds candidate_multi_x2 = 2 * multi_harmonics_peak
#   - Outputs:
#       * Heatmap PNGs per window
#       * Scatter PNGs for strongest pairs per window (identity + regression)
#       * One multi-page PDF
#       * Excel sheet "analysis4_correlations" in stat_results.xlsx
# ============================================================
import os
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import pearsonr, spearmanr
from openpyxl import load_workbook

# ------------------ PATHS ------------------
INPUT_FILES = [
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window8.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window15.txt",
    r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\final_results_window30.txt",
]
OUTPUT_DIR   = r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\figures_ANALYSIS4"
OUTPUT_XLSX  = r"C:\Users\sofia\OneDrive\Escritorio\TFGPython\statistics\stat_results.xlsx"
SHEET_NAME   = "analysis4_correlations"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------ STYLE (large & readable) ------------------
plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 18,
    "axes.titlesize": 24,
    "axes.labelsize": 20,
    "legend.fontsize": 16,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
})

# ------------------ TXT COLUMNS ------------------
COLS = [
    "patient","fragment","start_time","unit","window_size_s","num_windows",
    "time_peak_to_peak_amp","first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"
]

# ------------------ CONFIG ------------------
# If True, adds a 5th metric: candidate_multi_x2 = 2 * multi_harmonics_peak
EXTENDED = False

# Strong-pair selection policy for scatters
TOP_K_PER_WINDOW = 3          # take top-K pairs by |Pearson r|
MIN_ABS_R        = 0.85       # require at least this absolute r

# ------------------ HELPERS ------------------
def ensure_dir_for(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)

def read_results_txt(path: str) -> pd.DataFrame:
    """
    Read TXT, coerce dtypes, sort for stable output.
    """
    df = pd.read_csv(path, comment="#", header=None, names=COLS)
    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
    num = ["window_size_s","num_windows","time_peak_to_peak_amp",
           "first_harmonic_peak","first_harmonic_p2p","multi_harmonics_peak"]
    df[num] = df[num].apply(pd.to_numeric, errors="coerce")
    df = df.sort_values(["patient","fragment","start_time"]).reset_index(drop=True)
    return df

def label_from_window(df: pd.DataFrame, filename: str) -> str:
    """
    Return clean label like '8s', '15s', '30s'.
    """
    if df["window_size_s"].notna().any():
        try:
            v = int(round(float(df["window_size_s"].dropna().iloc[0])))
            return f"{v}s"
        except Exception:
            pass
    for cand in ("8","15","30"):
        if cand in os.path.basename(filename):
            return f"{cand}s"
    return "unknown"

def simple_linear_regression(x, y):
    """
    Fit y = a + b*x over finite pairs only. Return (a, b, R^2).
    """
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if x.size < 2:
        return np.nan, np.nan, np.nan
    b, a = np.polyfit(x, y, 1)  # slope=b, intercept=a
    yhat = a + b * x
    ss_res = float(np.sum((y - yhat)**2))
    ss_tot = float(np.sum((y - np.nanmean(y))**2))
    r2 = np.nan if ss_tot == 0 else 1 - ss_res/ss_tot
    return a, b, r2

def compute_pairwise_corr(df_metrics: pd.DataFrame):
    """
    Compute Pearson r (and p) and Spearman rho (and p) for all pairs.
    Returns:
        - corr_matrix_pearson: square DataFrame with Pearson r
        - pairs_long: long table with rows per pair (i<j) containing stats
    """
    cols = list(df_metrics.columns)
    # Pearson matrix with pairwise complete observations
    r_mat = pd.DataFrame(np.eye(len(cols)), index=cols, columns=cols, dtype=float)
    pairs_rows = []

    for i, j in itertools.combinations(range(len(cols)), 2):
        xi = df_metrics.iloc[:, i].to_numpy(dtype=float)
        yj = df_metrics.iloc[:, j].to_numpy(dtype=float)
        m  = np.isfinite(xi) & np.isfinite(yj)
        n  = int(m.sum())
        if n >= 2:
            r, p  = pearsonr(xi[m], yj[m])
            rh, ph = spearmanr(xi[m], yj[m])
        else:
            r = p = rh = ph = np.nan

        r_mat.iloc[i, j] = r
        r_mat.iloc[j, i] = r

        pairs_rows.append({
            "metric_x": cols[i],
            "metric_y": cols[j],
            "n": n,
            "pearson_r": r,
            "pearson_p": p,
            "spearman_rho": rh,
            "spearman_p": ph,
        })

    corr_matrix_pearson = r_mat
    pairs_long = pd.DataFrame(pairs_rows)
    return corr_matrix_pearson, pairs_long

def plot_heatmap(corr_df: pd.DataFrame, title: str, out_path: str):
    """
    Simple heatmap with imshow and value annotations (no external libs).
    """
    fig, ax = plt.subplots(figsize=(12, 10))
    data = corr_df.to_numpy(dtype=float)
    im = ax.imshow(data, vmin=-1, vmax=1)  # default colormap

    # Ticks / labels
    ax.set_xticks(range(corr_df.shape[1]))
    ax.set_yticks(range(corr_df.shape[0]))
    ax.set_xticklabels(corr_df.columns, rotation=45, ha="right")
    ax.set_yticklabels(corr_df.index)

    # Grid lines (light)
    ax.set_xticks(np.arange(-0.5, corr_df.shape[1], 1), minor=True)
    ax.set_yticks(np.arange(-0.5, corr_df.shape[0], 1), minor=True)
    ax.grid(which="minor", linestyle="-", linewidth=0.5, alpha=0.5)

    # Annotate r values
    for i in range(corr_df.shape[0]):
        for j in range(corr_df.shape[1]):
            val = data[i, j]
            if np.isfinite(val):
                ax.text(j, i, f"{val:.2f}", ha="center", va="center", fontsize=12)

    ax.set_title(title)
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.savefig(out_path, bbox_inches="tight")
    plt.close()

def place_legend_below(ax, ncol=2):
    """
    Place legend outside, centered below, with extra bottom margin.
    """
    plt.subplots_adjust(bottom=0.28)
    ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=True)

def safe_name(s: str) -> str:
    """Make a safe filename fragment from a metric name."""
    return s.replace(" ", "").replace("/", "_").replace("*", "x")

# ------------------ MAIN ------------------
png_paths = []
all_pairs_long = []          # long table across all windows
all_top_pairs_summary = []   # which pairs were plotted per window
corr_matrices_blocks = []    # (window_label, corr_matrix_pearson)

for path in INPUT_FILES:
    df = read_results_txt(path)
    win_label = label_from_window(df, path)

    # --- Choose metrics ---
    metrics = [
        "time_peak_to_peak_amp",
        "first_harmonic_peak",
        "first_harmonic_p2p",
        "multi_harmonics_peak",
    ]
    df_metrics = df[metrics].copy()
    if EXTENDED:
        df_metrics["candidate_multi_x2"] = 2.0 * df["multi_harmonics_peak"]

    # --- Preview to verify columns ---
    print("\n" + "-"*90)
    print(f"File: {os.path.basename(path)} | window: {win_label}")
    print("Preview of metrics (first 5 rows):")
    print(df_metrics.head(5).to_string(index=False))

    # --- Correlations ---
    corr_matrix_pearson, pairs_long = compute_pairwise_corr(df_metrics)
    pairs_long["window_label"] = win_label
    all_pairs_long.append(pairs_long)
    corr_matrices_blocks.append((win_label, corr_matrix_pearson))

    # --- Save heatmap ---
    heatmap_png = os.path.join(OUTPUT_DIR, f"heatmap_Pearson_{win_label}.png")
    plot_heatmap(corr_matrix_pearson, f"Correlation (Pearson) — {win_label}", heatmap_png)
    png_paths.append(heatmap_png)

    # --- Choose strongest pairs for scatters (by |r|) ---
    # Filter by minimum |r| and n>=2, sort desc by |r|
    pl = pairs_long.copy()
    pl = pl[np.isfinite(pl["pearson_r"])]
    pl = pl[pl["n"] >= 2]
    pl["abs_r"] = pl["pearson_r"].abs()
    pl = pl.sort_values("abs_r", ascending=False)
    pl = pl[pl["abs_r"] >= MIN_ABS_R]
    top_pairs = pl.head(TOP_K_PER_WINDOW)

    # --- Generate scatter for selected pairs ---
    for _, row in top_pairs.iterrows():
        mx, my = row["metric_x"], row["metric_y"]
        x = df_metrics[mx].to_numpy(dtype=float)
        y = df_metrics[my].to_numpy(dtype=float)
        mask = np.isfinite(x) & np.isfinite(y)
        n = int(mask.sum())

        if n < 2:
            continue

        r, p = pearsonr(x[mask], y[mask])
        a, b, r2 = simple_linear_regression(x, y)

        # axis limit
        lim = float(np.nanmax([np.nanmax(x[mask]), np.nanmax(y[mask]), 0.0]))
        lim = lim * 1.05 if np.isfinite(lim) and lim > 0 else 1.0

        fig = plt.figure(figsize=(16, 9))
        ax = plt.gca()

        ax.scatter(x, y, s=70, label="Fragments")
        # Identity line
        ax.plot([0, lim], [0, lim], linestyle="--", linewidth=1, label="Identity")
        # Regression line
        if not (math.isnan(a) or math.isnan(b)):
            xline = np.linspace(0, lim, 200)
            yline = a + b * xline
            ax.plot(xline, yline, linestyle="-", linewidth=1, label="Linear fit")

        ax.set_xlabel(mx)
        ax.set_ylabel(my)
        ax.set_title(f"{mx} vs {my} — {win_label}")
        ax.grid(True, alpha=0.3)
        place_legend_below(ax, ncol=2)

        txt = (f"n = {n}\n"
               f"Pearson r = {r:.3f}, p = {p:.3g}\n"
               f"Fit: y = a + b·x   a = {a:.3f},  b = {b:.3f}\n"
               f"R² = {r2:.3f}")
        # Add small text box
        fig.text(0.02, 0.04, txt, fontsize=16,
                 bbox=dict(facecolor="white", alpha=0.95, edgecolor="black", boxstyle="round,pad=0.5"))

        out_png = os.path.join(
            OUTPUT_DIR, f"scatter_{safe_name(mx)}_vs_{safe_name(my)}_{win_label}.png"
        )
        plt.savefig(out_png, bbox_inches="tight"); plt.close()
        png_paths.append(out_png)

        # Keep a summary row for the Excel sheet
        all_top_pairs_summary.append({
            "window_label": win_label,
            "metric_x": mx,
            "metric_y": my,
            "n": n,
            "pearson_r": r,
            "pearson_p": p,
            "linreg_intercept": a,
            "linreg_slope": b,
            "linreg_r2": r2,
        })

# ------------------ MULTI-PAGE PDF ------------------
pdf_path = os.path.join(OUTPUT_DIR, "analysis4_heatmaps_and_scatters.pdf")
with PdfPages(pdf_path) as pdf:
    for p in png_paths:
        img = plt.imread(p)
        plt.figure(figsize=(11.69, 8.27))  # A4 landscape
        plt.imshow(img)
        plt.axis("off")
        pdf.savefig(bbox_inches="tight")
        plt.close()

print("Figures saved in:", OUTPUT_DIR)
print("PDF multipage:", pdf_path)

# ------------------ WRITE EXCEL (single sheet for Analysis 4) ------------------
ensure_dir_for(OUTPUT_XLSX)

# Remove sheet if exists (clean start)
if os.path.exists(OUTPUT_XLSX):
    wb = load_workbook(OUTPUT_XLSX)
    if SHEET_NAME in wb.sheetnames:
        wb.remove(wb[SHEET_NAME])
        wb.save(OUTPUT_XLSX)
    wb.close()
    mode = "a"
else:
    mode = "w"

pairs_long_all = pd.concat(all_pairs_long, ignore_index=True)
top_pairs_df   = pd.DataFrame(all_top_pairs_summary)

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl", mode=mode, if_sheet_exists="overlay") as writer:
    # 1) Long table with all pairs across windows
    pairs_long_all.to_excel(writer, sheet_name=SHEET_NAME, index=False)

    ws = writer.sheets[SHEET_NAME]
    startrow = pairs_long_all.shape[0] + 2  # blank line

    # 2) One block per window with the Pearson corr matrix
    for win_label, cm in corr_matrices_blocks:
        ws.cell(row=startrow, column=1, value=f"CORR_MATRIX (Pearson) — {win_label}")
        cm.to_excel(writer, sheet_name=SHEET_NAME, startrow=startrow + 2)
        startrow += cm.shape[0] + 4  # matrix rows + title+blank

    # 3) Summary of top pairs plotted per window
    ws.cell(row=startrow, column=1, value="TOP_PAIRS_BY_WINDOW (used for scatters)")
    top_pairs_df.to_excel(writer, sheet_name=SHEET_NAME, startrow=startrow + 2, index=False)

print(f"\nSaved Excel: {OUTPUT_XLSX}  [sheet: {SHEET_NAME}]")

# ------------------ NOTES ------------------
# - To include the extended metric candidate_multi_x2 in the correlation maps,
#   set EXTENDED = True near the top of this script.
# - You can tune TOP_K_PER_WINDOW and MIN_ABS_R to control how many scatter
#   plots are generated and how strong the correlations should be.



------------------------------------------------------------------------------------------
File: final_results_window8.txt | window: 8s
Preview of metrics (first 5 rows):
 time_peak_to_peak_amp  first_harmonic_peak  first_harmonic_p2p  multi_harmonics_peak
                 0.800                0.259               0.517                 0.271
                 0.787                0.246               0.491                 0.263
                 0.727                0.247               0.493                 0.261
                 6.255                1.925               3.850                 2.033
                10.456                3.464               6.929                 3.642

------------------------------------------------------------------------------------------
File: final_results_window15.txt | window: 15s
Preview of metrics (first 5 rows):
 time_peak_to_peak_amp  first_harmonic_peak  first_harmonic_p2p  multi_harmonics_peak
                 0.799                0.227         